# IBM Instance Setup

In [4]:
from dotenv import load_dotenv
import os
import jax
import jax.numpy as jnp
import iqpopt as iqp
from iqpopt.utils import initialize_from_data, local_gates
import iqpopt.gen_qml as genq
from iqpopt.gen_qml.utils import median_heuristic
from utils.nisq import aachen_connectivity, efficient_connectivity_gates
from qiskit_ibm_runtime import QiskitRuntimeService
from qiskit_ibm_runtime import Session
import pennylane as qml
from datasets.bipartites import BipartiteGraphDataset
from datasets.er import ErdosRenyiGraphDataset
import numpy as np

key = jax.random.PRNGKey(42)

# Experiments Poster

In [5]:
NODES = 8
SP = "Sparse"
MD = "Medium"
DS = "Dense"

ER = "ER"
BP = "Bipartite"

In [6]:
ER8NSParams = np.load("results/params/params_8N_ER_Sparse_LR0.04522709518095061_SIGMA0.6587519702447917_INIT0.17944082088572766_NUMLAYERS1.npy")
ER8NMParams = np.load("results/params/params_8N_ER_Medium_LR0.011220685773844122_SIGMA0.6962179567887462_INIT1.810103830663358_NUMLAYERS5.npy")
ER8NDParams = np.load("results/params/params_8N_ER_Dense_LR0.04206675941389984_SIGMA0.7206244967648459_INIT0.5521825859396434_NUMLAYERS1.npy")
BP8NSParams = np.load("results/params/params_8N_Bipartite_Sparse_LR0.042174583320667126_SIGMA0.8175072163174264_INIT1.0464881642904231_NUMLAYERS1.npy")
BP8NMParams = np.load("results/params/params_8N_Bipartite_Medium_LR0.003551614405932626_SIGMA1.3352557243570238_INIT0.8853914111571922_NUMLAYERS5.npy")
BP8NDParams = np.load("results/params/params_8N_Bipartite_Dense_LR0.0018973825577720538_SIGMA1.3106607895258173_INIT0.1942402847384792_NUMLAYERS3.npy")

# Hardware test

In [7]:
load_dotenv('.env')

ibm_token = os.getenv('IBM_TOKEN')
instance = os.getenv("INSTANCE")
setup = True 

if setup:
    QiskitRuntimeService.save_account(channel="ibm_quantum", token=ibm_token, overwrite=True)

service = QiskitRuntimeService(channel="ibm_cloud", token = ibm_token, instance=instance)
backend = service.least_busy(operational=True, simulator=False, min_num_qubits=150)
print(backend)

<IBMBackend('ibm_aachen')>


In [8]:
dev = qml.device('qiskit.remote', 
                    wires=backend.num_qubits, 
                    backend=backend, 
                    shots=512,
                    optimization_level=3)

## Experiments

In [9]:
QUBITS = NODES * (NODES - 1) // 2
grid_conn = aachen_connectivity()

L1G = efficient_connectivity_gates(grid_conn, QUBITS, 1)
L2G = efficient_connectivity_gates(grid_conn, QUBITS, 2)
L3G = efficient_connectivity_gates(grid_conn, QUBITS, 3)
L4G = efficient_connectivity_gates(grid_conn, QUBITS, 4)
L5G = efficient_connectivity_gates(grid_conn, QUBITS, 5)

L1IQP = iqp.IqpSimulator(QUBITS, L1G, device="lightning.qubit")
L2IQP = iqp.IqpSimulator(QUBITS, L2G, device="lightning.qubit")
L3IQP = iqp.IqpSimulator(QUBITS, L3G, device="lightning.qubit")
L4IQP = iqp.IqpSimulator(QUBITS, L4G, device="lightning.qubit")
L5IQP = iqp.IqpSimulator(QUBITS, L5G, device="lightning.qubit")

@qml.qnode(dev)
def L1Sample(trained_params):
    L1IQP.iqp_circuit(np.asarray(trained_params))
    return qml.sample(wires = range(QUBITS))

@qml.qnode(dev)
def L2Sample(trained_params):
    L2IQP.iqp_circuit(np.asarray(trained_params))
    return qml.sample(wires = range(QUBITS))

@qml.qnode(dev)
def L3Sample(trained_params):
    L3IQP.iqp_circuit(np.asarray(trained_params))
    return qml.sample(wires = range(QUBITS))

@qml.qnode(dev)
def L4Sample(trained_params):
    L4IQP.iqp_circuit(np.asarray(trained_params))
    return qml.sample(wires = range(QUBITS))

@qml.qnode(dev)
def L5Sample(trained_params):
    L5IQP.iqp_circuit(np.asarray(trained_params))
    return qml.sample(wires = range(QUBITS))

In [10]:
with Session(backend=backend, max_time="12h") as session:
    ER8NSRes  = L1Sample(ER8NSParams)
    np.save("results/samples/2025-06-06/ER8NS.npy", ER8NSRes)
    ER8NMRes  = L5Sample(ER8NMParams)
    np.save("results/samples/2025-06-06/ER8NM.npy", ER8NMRes)
    ER8NDRes  = L1Sample(ER8NDParams)
    np.save("results/samples/2025-06-06/ER8ND.npy", ER8NDRes)
    BP8NDRes = L3Sample(BP8NDParams)
    np.save("results/samples/2025-06-06/BP8ND.npy", BP8NDRes)
    BP8NMRes = L5Sample(BP8NMParams)
    np.save("results/samples/2025-06-06/BP8NM.npy", BP8NMRes)
    BP8NSRes = L1Sample(BP8NSParams)
    np.save("results/samples/2025-06-06/BP8NS.npy", BP8NSRes)

In [11]:
print("Samples saved successfully.")

Samples saved successfully.
